In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/db.15.50.28
(please check that this page displays the message 'fluffy cats sit quickly')


Check we're logged in

In [ ]:
user.wait_for_login()

True

In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

hugs = Service(service_url="%s/hugs" % base_url)

creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
def upload_file(authenticated_user, filepath):
    """ Upload a file to the object store
    
        Args:
            filepath (str): Path to file to upload
        Returns:
            None    
    """
    creds = StorageCreds(user=authenticated_user, service_url="%s/storage" % base_url)
    drive = Drive(creds=creds, name="test_drive")
    filemeta = drive.upload(filepath)
    par = PAR(location=filemeta.location(), user=user)

    par_secret = hugs.encrypt_data(par.secret())
    auth = Authorisation(resource="process", user=user)

In [ ]:
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

### Upload files for processing


Upload Bilsdale data

In [ ]:
bsd_path = get_CRDS_path("bsd.picarro.1minute.248m.dat")
upload_file(authenticated_user=user, filepath=bsd_path)


Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
response = hugs.call_function(function="clear_datasources", args={})

In [ ]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}


response = hugs.call_function(function="process", args=args)

In [ ]:
print(response)

In [ ]:
from HUGS.Client import Process

filename = "bsd.picarro.1minute.248m.dat"
filepath = get_CRDS_path(filename)

processing = Process(service_url=base_url)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'bsd.picarro.1minute.248m.dat': ['c3b8313f-ebde-4bb3-8027-d11bdb932930', '4e8022b6-d458-4e3d-9efd-ee0931151c85', 'b80f2f24-0192-4b8d-a213-0b0543ead5d1']}


Upload some Heathfield data

In [ ]:
filename = "hfd.picarro.1minute.100m_min.dat"
filepath = get_CRDS_path(filename)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'hfd.picarro.1minute.100m_min.dat': ['5f54eac9-0e22-4480-a007-f9950abb14e3', 'b4dd7549-8040-4fd3-ab4e-b6d0f78c17e4', '5c5d1300-f1ae-43ef-8eb1-7997ef4e1559']}


In [ ]:
from HUGS.Processing import search

search_results = None

search_terms = widgets.Text(
                    value="",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

start_picker = widgets.DatePicker(
                    description='Start date',
                    disabled=False
)

end_picker = widgets.DatePicker(
                    description='End date',
                    disabled=False
)


box = widgets.VBox(children=[search_terms, locations, start_picker, end_picker, data_type,
                                 search_button], layout=layout)



def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    from HUGS.Client import Search
    start = datetime(1970, 1,1) # datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.now() # datetime.combine(end_picker.value, datetime.min.time())
    
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    split_locations = locations.value.replace(" ", "").split(",")
    
    global search_results
    search = Search(service_url=base_url)
    search_results = search.search(search_terms=split_search_terms, locations=split_locations, data_type=data_type.value, start_datetime=start, end_datetime=end)

search_button.on_click(call_search)




In [ ]:
box

['co2'] ['bsd']


In [ ]:
print(search_results)

{'bsd_co2': ['data/uuid/7dcb8db4-d0a3-427b-a23e-7ea8d5352489/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/03b2d164-2502-420f-823f-e2f1f8e8b61d/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/07fcaa49-ca3e-4b24-ada7-5ca1c9010958/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/ff21298c-2fc5-4a33-81cf-a00f0cc9e236/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/4e8022b6-d458-4e3d-9efd-ee0931151c85/2014-01-30T10:52:30_2014-01-30T14:20:30']}


Now take the result dictionary, parse the dictionary into a blocks that can be selected by the user

In [ ]:
def parse_results(results):
    """ Split the keys into a list of each key and the date that the data covers
        
        Args:
            results (dict): Dictionary of search results
        Returns:
            list (tuple): List of date, data key list pairs
    """
    date_keys = {}
    for key in results.keys():
        keys = sorted(results[key])
        start_key = keys[0]
        end_key = keys[-1]
        # Get the first and last dates from the keys in the search results
        start_date = start_key.split("/")[-1].split("_")[0]
        end_date = end_key.split("/")[-1].split("_")[-1]
        
        dates_covered = start_date + "_" + end_date
        
        date_keys[key] = {"dates": dates_covered, "keys": keys}
        
    return date_keys

Site, gas, dates and checkbox

In [ ]:
date_keys = parse_results(search_results)
print(date_keys)

{'bsd_co2': {'dates': '2014-01-30T10:52:30_2014-01-30T14:20:30', 'keys': ['data/uuid/03b2d164-2502-420f-823f-e2f1f8e8b61d/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/07fcaa49-ca3e-4b24-ada7-5ca1c9010958/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/4e8022b6-d458-4e3d-9efd-ee0931151c85/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/7dcb8db4-d0a3-427b-a23e-7ea8d5352489/2014-01-30T10:52:30_2014-01-30T14:20:30', 'data/uuid/ff21298c-2fc5-4a33-81cf-a00f0cc9e236/2014-01-30T10:52:30_2014-01-30T14:20:30']}}


In [ ]:
checkbox_objects = []
search_keys = []
date_labels = []
gas_labels = []
site_labels = []

for key in date_keys:
    checkbox_objects.append(widgets.Checkbox(value=False, description=key))
    search_keys.append(key)
    
    # Get the dates covered
    dates = date_keys[key]["dates"].replace("_", " ").replace("T", " ")
    date_label = widgets.Label(value=dates)
        
    split_key = key.split("_")
    site_name = split_key[0].upper()
    gas_name = split_key[1].upper()
    
    gas_label = widgets.Label(value=gas_name)
    site_label = widgets.Label(value=site_name)
    
    checkboxes.append((key, checkbox))
    date_labels.append(date_label)
    site_labels.append(site_label)
    gas_labels.append(gas_label)

# Create a dictionary of arguments that will be passed to the select_data function
arg_dict = {search_keys[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

checkbox_ui = widgets.VBox(children=checkbox_objects)
gas_ui = widgets.VBox(children=gas_labels)
site_ui = widgets.VBox(children=site_labels)
dates_ui = widgets.VBox(children=date_labels)

title_ui = widgets.HBox(children=[widgets.Label(value="Site"), widgets.Label(value="Gas"), widgets.Label(value="Date range"), widgets.Label(value="Select")], layout = widgets.Layout(flex_flow='row wrap'))
data_ui = widgets.HBox(children=[site_ui, gas_ui, dates_ui, checkbox_ui], layout = widgets.Layout(flex_flow='row wrap'))

ui = widgets.VBox(children=[title_ui, data_ui])

def select_data(**kwargs):
    selected_data.clear()
    
    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)
    
    print(selected_data)

out = widgets.interactive_output(select_data, arg_dict)
display(ui, out)
    

In [ ]:
from ipywidgets import Checkbox, HTML, Label, HBox, VBox

table_style = {'description_width': 'initial'}
table_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
date_layout = {'width':'275px', 'min_width':'200px', 'height':'28px', 'min_height':'28px'}
checkbox_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}

# row_layout = {'width':'200px', 'min_width':'200px'}

header_label_site = HTML(value=f"<b>Site</b>", layout=table_layout)
header_label_gas = HTML(value=f"<b>Gas</b>", layout=table_layout)
header_label_dates = HTML(value=f"<b>Dates</b>", layout=date_layout)
header_label_select = HTML(value=f"<b>Select</b>", layout=checkbox_layout)

checkbox_objects = []
search_keys = []

site_labels = []
date_labels = []
gas_labels = []
for key in date_keys:
    # Create the checkboxes
    checkbox = Checkbox(value=False, layout=checkbox_layout)
    checkbox_objects.append(checkbox)
    search_keys.append(key)
    
    dates = date_keys[key]["dates"].replace("_", " to ").replace("T", " ")
    date_label = Label(value=dates, layout=date_layout)
    
    split_key = key.split("_")
    site_name = split_key[0].upper()
    gas_name = split_key[1].upper()
    
    gas_label = Label(value=gas_name, layout=table_layout)
    site_label = Label(value=site_name, layout=table_layout)

    date_labels.append(date_label)
    site_labels.append(site_label)
    gas_labels.append(gas_label)

    
arg_dict = {search_keys[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

header_box = HBox(children=[header_label_site, header_label_gas, header_label_dates, header_label_select])

site_vbox = VBox(children=site_labels)
gas_vbox = VBox(children=gas_labels)
dates_vbox = VBox(children=date_labels)
checkbox_vbox = VBox(children=checkbox_objects)

dynamic_box = HBox(children=[site_vbox, gas_vbox, dates_vbox, checkbox_vbox])

complete = VBox(children=[header_box, dynamic_box])

selected_data = []
def select_data(**kwargs):
    selected_data.clear()
    
    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)
    
    print(selected_data)

out = widgets.interactive_output(select_data, arg_dict)

display(complete, out)

Output()

Retrieve the data from the object store

In [ ]:
selected_data

['bsd_co2']

In [ ]:
from HUGS.Client import Retrieve

retrieve = Retrieve(service_url=base_url)

keys = {key: date_keys[key]["keys"] for key in selected_data}
result = retrieve.retrieve(keys=keys)


ValueError: Error calling 'retrieve' on 'https://hugs.acquire-aaai.com/t/hugs': Dataframe index is not unique

Plot the data we've selected

Now we can plot some of the data

In [ ]:
import pandas as pd
# co2_data = pd.read_json(data["results"]["bsd_co2"])
co_data = pd.read_json(data["results"]["bsd_ch4"])


In [ ]:
co_data.head(10)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig = plt.figure()
# ax = fig.add_subplot(211)
# ax.plot(co2_data.index.values, co2_data["co2 count"], color = "#4e79a7" )
ax = fig.add_subplot(111)
ax.plot(co_data.index.values, co_data["ch4 count"], color = "#59a14f")

plt.show()

In [ ]:
# Checkboxes for selecting which data we want to plot
# Select the gas, it goes and plots the 
# Simple line plot

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [54.2361, -4.548]
zoom = 5
m = Map(center=center, zoom=zoom)
mark_bsd = Marker(location=(54.942544, -1.369204))
mark_bsd.popup = HTML(value="Bilsdale")

mark_mhd = Marker(location=(53.20,-9.54))
mark_mhd.popup = HTML(value="Macehead")

mark_tac = Marker(location=(52.511, 1.155003))
mark_tac.popup = HTML(value="Tacolneston")

m += mark_bsd
m += mark_mhd
m += mark_tac

display(m)

In [ ]:
m